# Contoh Penggunaan Alat AutoGen


## Import Pakej yang Diperlukan


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Membuat Klien

Dalam contoh ini, kita akan menggunakan [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) untuk akses kepada LLM.

`model` ditetapkan sebagai `gpt-4o-mini`. Cuba tukar model kepada model lain yang tersedia di pasaran GitHub Models untuk melihat hasil yang berbeza.

Sebagai ujian ringkas, kita hanya akan menjalankan arahan mudah - `Apakah ibu kota Perancis`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Mendefinisikan Fungsi

Dalam contoh ini, kami akan memberikan ejen akses kepada alat yang merupakan fungsi dengan senarai destinasi percutian yang tersedia dan ketersediaannya.

Anda boleh bayangkan bahawa ini adalah situasi di mana ejen pelancongan mungkin mempunyai akses kepada pangkalan data pelancongan, sebagai contoh.

Semasa anda melalui sampel ini, jangan ragu untuk mencuba mendefinisikan fungsi dan alat baharu yang boleh dipanggil oleh ejen.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Mendefinisikan Alat Fungsi
Untuk membolehkan ejen menggunakan `vacation_destinations` sebagai `FunctionTool`, kita perlu mendefinisikannya sebagai satu.

Kita juga akan memberikan penerangan tentang alat tersebut yang berguna untuk ejen mengenal pasti kegunaan alat itu berkaitan dengan tugas yang diminta oleh pengguna.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Mendefinisikan ejen

Sekarang kita boleh mencipta ejen dalam kod di bawah. Kami mendefinisikan `system_message` untuk memberikan arahan kepada ejen tentang cara membantu pengguna mencari destinasi percutian.

Kami juga menetapkan parameter `reflect_on_tool_use` kepada true. Ini membolehkan LLM menggunakan respons daripada panggilan alat dan menghantar respons menggunakan bahasa semula jadi.

Anda boleh menetapkan parameter kepada false untuk melihat perbezaannya.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Menjalankan Ejen

Sekarang kita boleh menjalankan ejen dengan mesej awal pengguna yang meminta untuk melancong ke Tokyo.

Anda boleh menukar destinasi bandar ini untuk melihat bagaimana ejen bertindak balas terhadap ketersediaan bandar tersebut.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
